In [ ]:
import pandas as pd
from csv import writer
from urllib.parse import urljoin
import unicodedata
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time
import requests
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException

In [ ]:
#to fix the ssleoferror

import requests
sess = requests.Session()
adapter = requests.adapters.HTTPAdapter(max_retries = 20)
sess.mount('http://', adapter)

In [ ]:
for j in range(397,476):  
    
    print('scraping page {}....'.format(j))
    res = sess.get('https://jamaicaclassifiedonline.com/?s=&category=cars&category=cars&type=&title=&desc=&price=&currency=&place=&parish=&recent=&sort=&hasimages=&orderby=modified_date&page={}'.format(j))

    soup = BeautifulSoup(res.text,'html.parser')

    car_el = soup.select('div.row>div[class="col l3 s12 m6"]>div>a')

    links = []
    for i in car_el:
        link = i.get('href')
        links.append(link)   
    
    
    
    with open('cars_jamclassified.csv','a',encoding ='utf8',newline = '') as file:
        writer_file = writer(file)
        header = ['ID','Title','Make','Model','Location','Year','Price','Phone','Additional_Info','Fuel Type','Doors','Transmission','Body Type','Driver Side','Condition']
        writer_file.writerow(header) 

        for k in links:
            print('scraping car link {}'.format(k))
            res = sess.get(k)

            soup = BeautifulSoup(res.text,'html.parser')
            try:
                title = soup.find('h1',attrs = {'id':'title'}).text.strip()
            except :
                title = "NA"

            try:
                location = soup.select('div.card-title>div>span')[0].text.strip()
            except :
                location = "NA"

            try:    
                price = soup.select('div.card-title>h1>span>span')[0].text.strip()
            except :
                price = "NA"

            try:
                phone = "'" + soup.find('a',attrs = {'href':'#modal-call'}).text.strip()
            except:
                phone = "NA"

            try:
                add_info = soup.select('div#item-description>div>div')[0].text.strip()
            except :
                add_info = "NA"

            lis = soup.select('ul.collection.with-header>li')

            ad_id =""
            year =""
            make=""
            model=""
            fuel_type =""
            doors =""
            transmission =""
            body_type=""
            driver_side=""
            condition ="" 

            for i in lis[2:]:


                obj = re.compile(r'ID:',re.I)
                match = obj.search(i.text)
                if match != None:
                    ad_id = i.text.strip()

                obj = re.compile(r'Year:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    year = i.text.strip()

                obj = re.compile(r'Make:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    make= i.text.strip()

                obj = re.compile(r'Model:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    model= i.text.strip()

                obj = re.compile(r'Fuel Type:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    fuel_type = i.text.strip()

                obj = re.compile(r'Doors:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    doors = i.text.strip()

                obj = re.compile(r'Transmission:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    transmission = i.text.strip()

                obj = re.compile(r'Body Type:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    body_type= i.text.strip()

                obj = re.compile(r'Driver Side:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    driver_side = i.text.strip()

                obj = re.compile(r'Condition:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    condition = i.text.strip()

            if ad_id == "":
                ad_id = 'NA'
            if year =="":
                year = 'NA'
            if make == "":
                make = 'NA'
            if model == "":
                model= 'NA'
            if fuel_type == "":
                fuel_type = 'NA'
            if doors == "":
                doors = 'NA'
            if transmission== "":
                transmission = 'NA'
            if body_type == "":
                body_type = 'NA'
            if driver_side == "":
                driver_side= 'NA'
            if condition == "":
                condition = 'NA'


            rows = [ad_id,title,make,model,location,year,price,phone,add_info,fuel_type,doors,transmission,body_type,driver_side,condition]

            writer_file.writerow(rows)

            print('Finished writing link {}, moving on to the next link'.format(k))
            
            time.sleep(2)
            
        print('Finished extracting Page {}'.format(j))

print('Done! Extracted all the information')


## Extracting Images for each cars

In [ ]:
for j in range(1,476):  
    
    print('scraping page {}....'.format(j))
    res = sess.get('https://jamaicaclassifiedonline.com/?s=&category=cars&category=cars&type=&title=&desc=&price=&currency=&place=&parish=&recent=&sort=&hasimages=&orderby=modified_date&page={}'.format(j))

    soup = BeautifulSoup(res.text,'html.parser')

    car_el = soup.select('div.row>div[class="col l3 s12 m6"]>div>a')

    links = []
    for i in car_el:
        link = i.get('href')
        links.append(link)   
        
    for i in links:
        print('getting images from {}'.format(i)'\n')
        res = sess.get(i)

        soup = BeautifulSoup(res.text,'html.parser')

        ad_id = soup.select('ul.collection.with-header>li:nth-child(2)>div')[0].text.strip()
        image = soup.select('div.col.s12.m8.l8 > div > div > div > a > img')


        try:  
            for i in image:

                im_el = i.get('src')
                with open('img_jamclassified.csv','a',encoding ='utf8',newline = "") as file:
                    writer_file = writer(file)
                    img_id= [ad_id,im_el]
                    writer_file.writerow(img_id)
                
        except:
            im_el = "NA"
            with open('img_jamclassified.csv','a',encoding ='utf8',newline = "") as file:
                writer_file = writer(file)
                img_id= [ad_id,im_el]
                writer_file.writerow(img_id)

print('Done! Got all the image links')
    

# Extracting House Ads from https://cbjamaica.com/

In [ ]:
for j in range(133,214):
    
    with open('houses_cbjam.csv','a',encoding ='utf8',newline = '') as file:
        
        writer_file = writer(file)
        header = ['ID','Title','Location','Latitude','Longitude','Price','Description','Up Price','Static Price','Down Price','Bedrooms','Bathrooms','Area','Interior','Exterior','Other','Site Influences']
        writer_file.writerow(header)  
        
        print('scraping page {}....'.format(j))

        res = sess.get('https://cbjamaica.com/property-search?page={}&limit=12&rent_sale=both&bed=&bath=&property_type=House&parish=&town=&currency=&keyword=&range=&agent=&agent_name=&similar=&tagOnly=&search_field=&idSearch=&property_amenities=&statuses=&price_range=&lattitude=&longitude='.format(j))

        soup = BeautifulSoup(res.text,'html.parser')

        car_el = soup.select('div#products>div>div>div:nth-child(3)>div>h2>a')

        links = []
        for i in car_el:
            link = i.get('href')
            links.append(link)



        for k in links:
            
            print('scraping car link {}'.format(k))
            res = sess.get(k)

            soup = BeautifulSoup(res.text,'html.parser')
            
            try:
                ad_id = soup.select('div.sale-offer-part > h1>span')[0].text.strip()
            except:
                ad_id = "NA"

            try:
                title = soup.select('div.sale-offer-part > h1')[0].text.strip()
                title = " ".join(title.split())  
            except:
                title = "NA"

            try:
                loc = soup.select('div.sale-offer-part>p')[0].text.strip()
                loc = " ".join(loc.split())
            except:
                loc = "NA"

            try:
                lat = soup.select('div#map-main')[0].get('data-lat')
            except:
                lat = "NA"
            try:
                lon = soup.select('div#map-main')[0].get('data-long')
            except:
                lon = "NA"

            try:
                price = soup.find('h2',attrs = {'class':'property-price'}).text.strip()
            except:
                price = "NA"

            try:
                descr = soup.find('div',attrs = {'class':'property-description'}).text.strip()
            except:
                descr = "NA"

            try:
                up_price = soup.find('li',attrs = {'class':'up-price'}).text.strip()
            except:
                up_price = "NA"

            try:
                static_price = soup.find('li',attrs = {'class':'static-price'}).text.strip()
            except:
                static_price = "NA"

            try:    
                down_price = soup.find('li',attrs = {'class':'down-price'}).text.strip()
            except:
                down_price = "NA"

            lis= soup.select('div.single-property-info-bottom>ul>li')

            bedrooms = ""
            bathrooms = ""
            area = ""

            for i in lis:
                obj = re.compile(r'Bedrooms',re.I)
                match = obj.search(i.text)
                if match != None:
                    bedrooms= i.text.strip()

                obj = re.compile(r'Bathrooms',re.I)
                match = obj.search(i.text)
                if match != None:
                    bathrooms= i.text.strip()

                obj = re.compile(r'Sq Ft',re.I)
                match = obj.search(i.text)
                if match != None:
                    area= i.text.strip()

            if bedrooms == "":
                bedrooms = 'NA'
            if bathrooms == "":
                bathrooms = 'NA'
            if area == "":
                area = 'NA'


            amen = soup.select('div.tab-inner-points>div>div>div')

            interior = ""
            exterior = ""
            other = ""
            site = ""

            for i in amen:
                obj = re.compile(r'Interior',re.I)
                match = obj.search(i.text)
                if match != None:
                    interior= i.text.strip()
                    interior = ", ".join(interior.split())

                obj = re.compile(r'Exterior',re.I)
                match = obj.search(i.text)
                if match != None:
                    exterior= i.text.strip()
                    exterior = ", ".join(exterior.split())

                obj = re.compile(r'Other',re.I)
                match = obj.search(i.text)
                if match != None:
                    other= i.text.strip()
                    other = ", ".join(other.split())

                obj = re.compile(r'Site Influences',re.I)
                match = obj.search(i.text)
                if match != None:
                    site= i.text.strip()
                    site = ", ".join(site.split())

            if interior == "":
                interior = 'NA'
            if exterior == "":
                exterior = 'NA'
            if other == "":
                other = 'NA'
            if site == "":
                site = 'NA'
                
            rows = [ad_id,title,loc,lat,lon, 
                    price,descr,up_price,static_price,down_price,bedrooms,bathrooms,area,interior,exterior,other,site]

            writer_file.writerow(rows)
            
            print('Finished writing link {}, moving on to the next link'.format(k))
            
            time.sleep(1)
            
        print('Finished extracting Page {}'.format(j))

print('Done! Extracted all the information')

In [ ]:
for j in range(1,214):
    
    with open('houses_location.csv','a',encoding ='utf8',newline = '') as file:
        
        writer_file = writer(file)
        header = ['ID','Location']
        writer_file.writerow(header)  
        
        print('scraping page {}....'.format(j))

        res = sess.get('https://cbjamaica.com/property-search?page={}&limit=12&rent_sale=both&bed=&bath=&property_type=House&parish=&town=&currency=&keyword=&range=&agent=&agent_name=&similar=&tagOnly=&search_field=&idSearch=&property_amenities=&statuses=&price_range=&lattitude=&longitude='.format(j))

        soup = BeautifulSoup(res.text,'html.parser')

        car_el = soup.select('div#products>div>div>div:nth-child(3)>div>h2>a')
 
        links = []
        for i in car_el:
            link = i.get('href')
            links.append(link)


        for k in links:
            print('scraping link {}'.format(k))
            
            res = sess.get(k)
            
            soup = BeautifulSoup(res.text,'html.parser')
            
            
            try:
                loc = soup.select('div.sale-offer-part>p')[0].text.strip()
                loc = " ".join(loc.split())
            except:
                loc = "NA"
                
            try:
                ad_id = soup.select('div.sale-offer-part>h1>span')[0].text.strip()
            except:
                ad_id = "NA"
                
            rows = [ad_id,loc]
            
            writer_file.writerow(rows)
            
print('Done! All locations extracted')
        


In [ ]:
res = sess.get('https://cbjamaica.com/property-details/MLS-59443')
soup = BeautifulSoup(res.text,'html.parser')

try:
    loc = soup.select('div.sale-offer-part>p')[0].text.strip()
    loc = " ".join(loc.split())
except:
    loc = "NA"

In [ ]:
links

# scraping House Images link for https://cbjamaica.com/

In [ ]:
for j in range(1,214):

    print('scraping page {}....'.format(j))

    res = sess.get('https://cbjamaica.com/property-search?page={}&limit=12&rent_sale=both&bed=&bath=&property_type=House&parish=&town=&currency=&keyword=&range=&agent=&agent_name=&similar=&tagOnly=&search_field=&idSearch=&property_amenities=&statuses=&price_range=&lattitude=&longitude='.format(j))

    soup = BeautifulSoup(res.text,'html.parser')

    car_el = soup.select('div#products>div>div>div:nth-child(3)>div>h2>a')

    links = []
    for i in car_el:
        link = i.get('href')
        links.append(link)
  
        
    for i in links:
        print('getting images from {}'.format(i))
        res = sess.get(i)

        soup = BeautifulSoup(res.text,'html.parser')

        try:
            ad_id = soup.select('div.sale-offer-part > h1>span')[0].text.strip()
        except:
            ad_id = "NA"
        
        image = soup.select('div.slick-track>img')


        try:  
            for i in image:

                im_el = i.get('src')
                with open('img_cbjam.csv','a',encoding ='utf8',newline = "") as file:
                    writer_file = writer(file)
                    img_id= [ad_id,im_el]
                    writer_file.writerow(img_id)
                
        except:
            im_el = "NA"
            with open('img_cbjam.csv','a',encoding ='utf8',newline = "") as file:
                writer_file = writer(file)
                img_id= [ad_id,im_el]
                writer_file.writerow(img_id)

                
                
print('Done! Got all the image links')
    

In [ ]:
import xlsxwriter
import pandas as pd

In [ ]:
workbook = xlsxwriter.Workbook('new.xlsx')
worksheet = workbook.add_worksheet()

name = ['ram','shyam','kohli']
runs = [34,45,67]

with open('new.xlsx') as file:
    
    worksheet.write(0,0,'names')



In [ ]:
df1 = pd.DataFrame({'dataset':['A', 'B', 'C', 'D', 'E']})
df2 = pd.DataFrame({'dataset':[3, 12, 18, 21]})

In [ ]:
with pd.ExcelWriter('dataframes.xlsx') as writer:
    
    df1.to_excel(writer, sheet_name = 'first', index = False)
    df2.to_excel(writer, sheet_name = 'second', index = False)

In [ ]:
df = pd.read_excel('dataframes.xlsx')

In [ ]:
df.head()

In [ ]:
df['comb'] = df['Prop Ad']+", " + df['City'] +", " +df['State']+ ", "  + df['County']

In [ ]:
df['comb']

In [ ]:
df['Zip']

## Getting data for car ads posted after 29.08.2022 - https://www.jacars.net/

In [ ]:
for j in range(10,11):
    
    res = sess.get('https://www.jacars.net/cars/?page={}&ordering=newest'.format(j))
    
    print('scraping page {}....'.format(j))

    soup = BeautifulSoup(res.text,'html.parser')


    other_el = soup.select('ul.list-simple__output.js-list-simple__output>li.announcement-container')



    master_link = []
    for i in other_el:
        
        date = i.select('div.list-announcement-block > div.announcement-block-text.announcement-block__text > div > div.announcement-block__date')[0]
        regexobj = re.compile(r'Today|Yesterday|31.08.2022|30.08.2022',re.I)

        car_date = date.text.strip()
        mat = regexobj.search(car_date)
        if mat != None :
            half_link = i.select('div.list-announcement-block > div.announcement-block-text.announcement-block__text > div > a')[0]
            full_link = urljoin('https://www.jacars.net',half_link.get('href'))
            master_link.append(full_link)

    
    
    with open('jacars_latest.csv','a',encoding ='utf8',newline = '') as file:
        writer_file = writer(file)
        header = ['ID','Title','address','price','phone','description','year','alarm','seats','airbag','colour','gearbox','mileage','body type','fuel type','drivetrain','engine size','air conditioning','right hand drive','power windows']
        writer_file.writerow(header)  
        
        
        for link in master_link:
            print('scraping car link {}....'.format(link))

            res = sess.get(link)

            soup = BeautifulSoup(res.text,'html.parser')

            ad_id = soup.find('span',attrs = {'class':'number-announcement'}).text.strip()

            car_title = soup.find('h1',attrs = {'id':'ad-title'}).text.strip()
            
            try:
                address = soup.find('span',attrs = {'itemprop':'address'}).text.strip()
            except NoSuchElementException:
                address = "NA"
            
            try:
                price = soup.find('div',attrs = {'class':'announcement-price__cost'}).text.strip().replace('\n                                \n                                  \n\n                                        \n                                          ',"")
            except NoSuchElementException:
                price = "NA"
                
            try:
                ph_no = soup.find('span',attrs = {'class':'phone-author-subtext__main'}).text.strip()
            except NoSuchElementException:
                ph_no = "NA"
                
            try:
                descr = soup.select('div.announcement-description > p')[0].text.strip()
            except:
                descr = "NA"
                
            lis = soup.select('ul.chars-column>li')
            
            year =""
            alarm =""
            seats=""
            airbag=""
            colour =""
            gearbox =""
            mileage =""
            body_type=""
            fuel_type=""
            drivetrain ="" 
            engine_size =""
            air_conditioning=""
            right_hand_drive= ""
            power_windows=""
            
            
            for i in lis:
                obj = re.compile(r'Year:',re.I)
                match = obj.search(i.text)
                if match != None:
                    year = i.text.strip()

                obj = re.compile(r'Alarm:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    alarm = i.text.strip()

                obj = re.compile(r'Seats:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    seats= i.text.strip()

                obj = re.compile(r'Airbag:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    airbag= i.text.strip()

                obj = re.compile(r'Colour:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    colour = i.text.strip()

                obj = re.compile(r'Gearbox:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    gearbox = i.text.strip()

                obj = re.compile(r'Mileage:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    mileage = i.text.strip()

                obj = re.compile(r'Body type:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    body_type= i.text.strip()

                obj = re.compile(r'Fuel type:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    fuel_type= i.text.strip()

                obj = re.compile(r'Drivetrain:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    drivetrain = i.text.strip()

                obj = re.compile(r'Engine size:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    engine_size = i.text.strip()

                obj = re.compile(r'Air conditioning:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    air_conditioning = i.text.strip()

                obj = re.compile(r'Right hand drive:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    right_hand_drive = i.text.strip()

                obj = re.compile(r'Power windows, doors:',re.I)
                match = obj.search(i.text)
                if match !=None:
                    power_windows= i.text.strip()
        
            if year == "":
                year = 'NA'
            if alarm =="":
                alarm = 'NA'
            if seats == "":
                seats = 'NA'
            if airbag == "":
                airbag = 'NA'
            if colour == "":
                colour = 'NA'
            if gearbox == "":
                gearbox = 'NA'
            if mileage== "":
                mileage = 'NA'
            if body_type == "":
                body_type = 'NA'
            if fuel_type == "":
                fuel_type= 'NA'
            if drivetrain == "":
                drivetrain = 'NA'
            if engine_size == "":
                engine_size = 'NA'
            if air_conditioning == "":
                air_conditioning = 'NA'
            if right_hand_drive== "":
                right_hand_drive = 'NA'
            if power_windows == "":
                power_windows= 'NA'

            
            rows = [ad_id,car_title,address,price,ph_no,descr,year,alarm,seats,airbag,colour,gearbox,mileage,body_type,fuel_type,drivetrain,engine_size,air_conditioning,right_hand_drive,power_windows]

            writer_file.writerow(rows)
            
            print('writing link {} finished, moving to next link'.format(link))
        
    print('Finished writing page {}, moving on to the next page'.format(j))

print('Done! Extracted all the information')


In [ ]:
date = other_el[0].select('div.list-announcement-block > div.announcement-block-text.announcement-block__text > div > div.announcement-block__date')[0]

## Getting image links for car ads posted after 29.08.2022 from https://www.jacars.net/

In [ ]:
for j in range(1,11):
    
    res = sess.get('https://www.jacars.net/cars/?page={}&ordering=newest'.format(j))
    
    print('scraping page {} ....'.format(j))

    soup = BeautifulSoup(res.text,'html.parser')


    other_el = soup.select('ul.list-simple__output.js-list-simple__output>li.announcement-container')



    master_link = []
    for i in other_el:
        
        date = i.select('div.list-announcement-block > div.announcement-block-text.announcement-block__text > div > div.announcement-block__date')[0]
        regexobj = re.compile(r'Today|Yesterday|31.08.2022|30.08.2022',re.I)

        car_date = date.text.strip()
        mat = regexobj.search(car_date)
        if mat != None :
            half_link = i.select('div.list-announcement-block > div.announcement-block-text.announcement-block__text > div > a')[0]
            full_link = urljoin('https://www.jacars.net',half_link.get('href'))
            master_link.append(full_link)

            
    for i in master_link:
        print('getting images from {}'.format(i))
        res = sess.get(i)

        soup = BeautifulSoup(res.text,'html.parser')

        ad_id = soup.find('span',attrs = {'class':'number-announcement'}).text.strip()
        
        image = soup.select('div.announcement__images>img[data-full]')

        try:  
            for i in image:

                im_el = i.get('src')
                with open('jacars_latest_images.csv','a',encoding ='utf8',newline = "") as file:
                    writer_file = writer(file)
                    img_id= [ad_id,im_el]
                    writer_file.writerow(img_id)
                
        except:
            im_el = "NA"
            with open('jacars_latest_images.csv','a',encoding ='utf8',newline = "") as file:
                writer_file = writer(file)
                img_id= [ad_id,im_el]
                writer_file.writerow(img_id)

print('Done! Got all the image links')

### Scraping https://iaccmanagement.govoffice2.com/index.asp?SEC=65D98D93-45C5-4634-83D5-1F7E6539A587&pri=0 (Iframe Switch involved)

In [ ]:
driver = webdriver.Edge()
driver.get('https://iaccmanagement.govoffice2.com/index.asp?SEC=65D98D93-45C5-4634-83D5-1F7E6539A587&pri=0')

iframe = driver.find_element_by_tag_name('iframe') #------------------->locate the iframe

driver.switch_to.frame(iframe) #--------------------> switch to iframe

emp_el = driver.find_elements_by_css_selector('tbody>tr>td')

for i in emp_el:
    name = i.find_element_by_css_selector('span>strong').text.strip()
    
    role = i.find_element_by_css_selector('span').text.strip()
    try:
        email = i.find_element_by_css_selector('a>span').text.strip()
    except:
        email = "NA"
    rows = [name,role,email]
    
    with open('employee.csv','a',encoding = 'utf8',newline = "") as file:
        writer_file  = writer(file)
        writer_file.writerow(rows)

In [ ]:
! pip install newspaper3k

In [ ]:
from newspaper import Article

In [ ]:
url = 'https://www.firstpost.com/opinion-news-expert-views-news-analysis-firstpost-viewpoint/congress-claim-on-ins-vikrant-is-spurious-nehru-rajiv-gandhi-used-navy-warships-aircraft-carriers-for-vacation-11168821.html'

In [ ]:
my_article = Article(url,language = "en")

In [ ]:
my_article.download()

In [ ]:
my_article.parse()

In [ ]:
my_article.keywords

In [ ]:
driver = webdriver.Edge()
url = 'https://www.google.com/maps/search/real+estate/@18.0482622,-77.7305469,9z/data=!3m1!4b1'

driver.get(url)

time.sleep(5)

#lis_el = driver.find_element_by_css_selector('div[aria-label="Results for real estate"]')
#driver.execute_script('arguments[0].scrollIntoView();',lis_el)



#lis_el = driver.find_element_by_css_selector('div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd')

#ActionChains(driver).move_to_element(lis_el).scroll




SCROLL_PAUSE_TIME = 3
last_height = driver.execute_script('return document.querySelector(\'[aria-label="Results for real estate"]\').scrollHeight')
print(last_height)

#driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight',lis_el)
while True:
    

   
    #Scroll down to bottom
    driver.execute_script('document.querySelector(\'[aria-label="Results for real estate"]\').scrollTo(0,1000000);')
    
    #Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    
   
    #print('ok')

    #Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script('return document.querySelector(\'[aria-label="Results for real estate"]\').scrollHeight')
    print(new_height)
    if new_height == last_height:
        break      
    last_height = new_height


lis_el = driver.find_elements_by_css_selector('div.m6QErb.DxyBCb.kA9KIf.dS8AEf.ecceSd > div>div[role="article"]')




In [ ]:
back_top_but = driver.find_element_by_css_selector('button[aria-label = "Back to top"]')
ActionChains(driver).click(back_top_but).perform()
time.sleep(3)

In [ ]:
len(lis_el)

In [ ]:
for i in lis_el:
    
    driver.execute_script('arguments[0].scrollIntoView();',i)
    
    

    ActionChains(driver).click(i).perform()

    time.sleep(3)
    
    name = driver.find_element_by_css_selector('h1.DUwDvf.fontHeadlineLarge>span').text
    print(name)

    try:
        addr = driver.find_element_by_css_selector('button[aria-label ^="Address"]').get_attribute('aria-label')
    except:
        addr = "NA"
        
    try:   
        website = driver.find_element_by_css_selector('a[aria-label ^="Website"]').get_attribute('href')
    except:
        website = "NA"
    
    try:
        phone = driver.find_element_by_css_selector('button[aria-label^="Phone"]').get_attribute('aria-label')
    except:
        phone = "NA"
        
    try:
        plus_code = driver.find_element_by_css_selector('button[aria-label^="Plus code"]').get_attribute('aria-label')
    except:
        plus_code = "NA"
    
    try:
        reviews = driver.find_element_by_css_selector('button[aria-label*="reviews"]').get_attribute('aria-label')
    except:
        reviews = "NA"
    
    try:
        stars = driver.find_element_by_css_selector('div[role = "button"]>span>span>span[aria-label*="stars"]').get_attribute('aria-label')
    except:
        stars = "NA"
    
    
    lis = [name,addr,website,phone,plus_code,reviews,stars]
    with open('real_estate.csv','a',encoding  = 'utf8',newline = "") as file:
        writer_file = writer(file)
        writer_file.writerow(lis)

In [ ]:
driver = webdriver.Edge()
url = 'https://www.google.com/maps/search/real+estate/@18.0482622,-77.7305469,9z/data=!3m1!4b1'

driver.get(url)

element = 